In [14]:
import os
import pathlib
import re
import json

raw_data_path = 'raw_data/app-insights'

In [16]:
qnsPattern = re.compile("question:.*")
queryPattern = re.compile('(?<=```)[^`]*(?=```)')
type = 'appinsights'
result = []

readmeQns = []
readmeQuery = []

for root, dirs, files in os.walk(raw_data_path, topdown=False):
    for f in files:
        ext = pathlib.Path(f).suffix
        if f != 'README.md':
            file = open(os.path.join(root, f), mode='r', encoding="utf8")
            content = file.read()
            file.close()
            qns = re.findall(qnsPattern, content)
            query = re.findall(queryPattern, content)

            result.append({
                'question': ''.join(qns).replace('question: ', ''),
                'queryType': 'appinsights',
                'query': ''.join(query)
            })

        elif f == 'README.md':
            qnsPattern = re.compile("(?<=###)[^`]*(?=\n)*")
            queryPattern = re.compile('(?<=```\n)[^```]*(?=```)')
            
            file = open(os.path.join(root, f), mode='r', encoding="utf8")
            content = file.read()
            file.close()

            questions = re.findall(qnsPattern, content)
            qMatch = re.findall(queryPattern, content)

            i = 0
            while i <= len(qMatch) - 1:    
                readmeQuery.append(qMatch[i])
                i += 2

            i = 0
            while i <= len(questions) - 1: 
                q = questions[i]
                spaceIdx = q.index('. ')
                readmeQns.append(q[spaceIdx + 2:])
                i += 1
            
            for idx in range(len(questions)):
                result.append({
                    'question': ''.join(readmeQns[idx]),
                    'queryType': 'appinsights',
                    'query': ''.join(readmeQuery[idx])
                })
            
            
with open('knowledge/app_insights_knowledge.json', 'w') as f:
    json.dump(result, f, indent=4)
            